# Demo Script for Milestone 10.15

## 0 Prepare task

### 0.1 prepare planning scene

#### Run shared detector on bash
```bash
python3 /home/kiro-ros/Projects/rnb-planning/src/scripts/demo_202107/demo_utils/shared_detector.py
```

#### Check and request ip setting from mobile udp client (robot-side)

#### 0.1.1 Set parameters and create planning scene

In [1]:
import numpy as np

CONNECT_INDY = True
CONNECT_MOBILE = True

CONNECT_TASK_PLANNER = False
VISUALIZE = False
VERBOSE = False
PLANNING_MULTIPROC = True
N_AGENTS = 10
NODE_TRIAL_MAX = N_AGENTS * 2
MAX_SOL_NUM = 5
BASE_COST_CUT = 110

TIMEOUT_MOTION = 0.5
TIMEOUT_FULL = 5

ROS_MASTER_ON_MOBILE = False
# Tool dimensions
TOOL_DIM = [0.24, 0.35]
TOOL_CENTER_OFF = [0, -0.035, 0]
TOOL_THICKNESS = 0.11 + 0.05

INDY_BASE_OFFSET = (0.172,0,0.439)
INDY_BASE_RPY = (0,0,0)
TOOL_NAME = "brush_face"
WALL_THICKNESS = 0.01
CLEARANCE = 0.001

COL_COLOR = (1,1,1,0.2)

IP_CUR = "192.168.0.10"# get_ip_address()
MOBILE_IP = "192.168.0.102"
INDY_IP = "192.168.0.3"

print("Current PC IP: {}".format(IP_CUR))
print("Mobile ROB IP: {}".format(MOBILE_IP))

Current PC IP: 192.168.0.10
Mobile ROB IP: 192.168.0.102


In [2]:
if CONNECT_TASK_PLANNER:
    from demo_proto.DisinfectionOperationServicer import serve_on_thread
    servicer = serve_on_thread()

In [3]:
import os
import sys
sys.path.append(os.path.join(os.path.join(
    os.environ["RNB_PLANNING_DIR"], 'src')))
sys.path.append(os.path.join(
    os.environ["RNB_PLANNING_DIR"], 'src/scripts/demo_202107'))

from pkg.global_config import RNB_PLANNING_DIR
from pkg.utils.utils import *    
from pkg.utils.rotation_utils import *
from pkg.controller.combined_robot import *
from demo_utils.area_select import *
from pkg.detector.aruco.marker_config import get_aruco_map
aruco_map = get_aruco_map()

from pkg.detector.multiICP.multiICP import *
from pkg.detector.detector_interface import DetectionLevel
from pkg.detector.multiICP.config import *

from pkg.geometry.builder.scene_builder import SceneBuilder
from demo_utils.environment import *
from demo_utils.area_select import DATASET_DIR, SweepDirections
from demo_utils.demo_config import *
from demo_utils.detection_util import *

from pkg.utils.shared_function import *
clear_channels_on("kiromobilemap")

# if not CONNECT_INDY:
#     indy_7dof_client.kiro_tool.KIRO_TOOL_PORT = '/dev/ttyS10'
kiro_udp_client.KIRO_UDP_OFFLINE_DEBUG = not CONNECT_MOBILE

mobile_config = RobotConfig(0, RobotType.kmb, ((0,0,0), (0,0,0)),
                "{}/{}".format(MOBILE_IP, IP_CUR))
robot_config = RobotConfig(1, RobotType.indy7kiro, 
                           (INDY_BASE_OFFSET, INDY_BASE_RPY),
                INDY_IP, root_on="kmb0_platform")
ROBOT_TYPE = robot_config.type
MOBILE_NAME = mobile_config.get_indexed_name()
ROBOT_NAME = robot_config.get_indexed_name()
crob = CombinedRobot(robots_on_scene=[mobile_config, robot_config]
              , connection_list=[True, CONNECT_INDY])

s_builder = SceneBuilder(None)
SceneBuilder.autostart_roscore = not ROS_MASTER_ON_MOBILE
gscene = s_builder.create_gscene(crob)

gtems = s_builder.add_robot_geometries(
    color=COL_COLOR, display=True, collision=True)
gscene.set_workspace_boundary(
    -1, 2.5, -0.5, 3, -CLEARANCE, 1.8, thickness=WALL_THICKNESS)


from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

ROBOT_BASE = pscene.robot_chain_dict[ROBOT_NAME]['link_names'][0]
TIP_LINK = pscene.robot_chain_dict[ROBOT_NAME]["tip_link"]
CAM_LINK = TIP_LINK.replace("tcp", "link6")
MOBILE_BASE = pscene.robot_chain_dict[MOBILE_NAME]["tip_link"]
HOLD_LINK = MOBILE_BASE

viewpoint = add_cam(gscene, tool_link=CAM_LINK, center=(-0.0785, 0, 0.073))

add_brush(gscene, face_name=TOOL_NAME, tool_link=TIP_LINK,
          thickness=TOOL_THICKNESS, tool_dim=TOOL_DIM,
          col_color=COL_COLOR, center_off=TOOL_CENTER_OFF)

HOME_POSE = crob.home_pose
HOME_DICT = list2dict(HOME_POSE, gscene.joint_names)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

# Set planner
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
from pkg.planning.filtering.grasp_filter import GraspChecker
mplan = MoveitPlanner(pscene, enable_dual=False, 
                      incremental_constraint_motion=True)
mplan.motion_filters = [GraspChecker(pscene)]
mplan.update_gscene()
gcheck = GraspChecker(pscene)
mplan.motion_filters = [gcheck]

mplan.reset_PRQdict(enable_PRQ=0.5, radii=5e-2)
for tip_dir, SWEEP_AXIS in [
    (SweepDirections.front, "X"), (SweepDirections.up, "Z"), (SweepDirections.down, "Z")]:
    filename = SweepDirections.get_file_name(ROBOT_TYPE, tip_dir.name+SWEEP_AXIS)+"-PRQ.pkl"
    PRQ_PATH = os.path.join(SWEEP_DATA_DIR, filename)
    try:
        Pos_Rotvec_Qlist_dict = load_pickle(PRQ_PATH)
        mplan.register_PRQ(ROBOT_NAME, Pos_Rotvec_Qlist_dict, decimal=2)
        print("Loaded: {}".format(filename))
    except:
        print("File not exist: {}".format(filename))
        continue

from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene, node_trial_max=NODE_TRIAL_MAX)
tplan.prepare()
ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

# Register binders
from pkg.planning.constraint.constraint_actor import VacuumTool, \
    Gripper2Tool, PlacePlane, SweepFramer

brush_face = pscene.create_binder(
    bname=TOOL_NAME, gname=TOOL_NAME, _type=SweepFramer, 
    point=(0,0, -gscene.NAME_DICT['brush_face'].dims[2]/2-CLEARANCE), 
    rpy=(0,0,0))

gscene.create_safe(
    gtype=GEOTYPE.BOX, name="floor_box", link_name="base_link",
    dims=(15,15,0.4), center=(0,0,0), rpy=(0,0,0), 
    color=(1, 1, 1, 0.1), display=True, collision=False, fixed=True)

gscene.add_highlight_axis("hl", "base_coord", T=np.identity(4), dims=(0.5,0.1,0.1))

kmb = crob.robot_dict["kmb0"]
indy = crob.robot_dict["indy1"]
mobile_box = gscene.NAME_DICT['kmb0_platform_Box_2']
crob.simulator.set_gscene(gscene)
# indy.ktool.led_on()
# indy.ktool.led_off()
# time.sleep(2)
# indy.ktool.send_degree(-45)

if CONNECT_MOBILE:
    assert np.sum(np.abs(get_xyzw_cur()))>1e-4, "UDP Server not connected"

connection command:
kmb0: True
indy1: True
[MOBILE ROBOT] bind: ('192.168.0.10', 50306)
[MOBILE ROBOT] Start UDP THREAD
goal reach: 0 -> 1 (1639572363.98)
goal reach signal edge up
==== Kiro Tool connected to /dev/ttyUSB0 (115200) ====
[KTOOL] initialize
[KTOOL] enable
[KTOOL] send (<MOTOR_CMD.ENABLE: 1>, <OP_CMD.NONE: 0>)
[KTOOL] recv �   �
[KTOOL] op_init
[KTOOL] send (<MOTOR_CMD.OP_INIT: 3>, <OP_CMD.NONE: 0>)
[KTOOL] recv ��  �
Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker
Loaded: indy7kiro-frontX-PRQ.pkl
Loaded: indy7kiro-upZ-PRQ.pkl
Loaded: indy7kiro-downZ-PRQ.pkl


#### 0.1.2 Load environment map

In [4]:
from demo_utils.ros_map_utils import KiroMobileMap
kmm = KiroMobileMap(MOBILE_IP, IP_CUR, CONNECT_MOBILE)
            
VALID_BOX_SCALE = 0.8
VALID_SCORE_CUT = 50
kmb.coster = (lambda Q: 
                  np.max(
                      kmm.get_box_costs(mobile_box, Q, kmm.T_bi, kmm.cost_im, kmm.resolution, 
                                        scale=VALID_BOX_SCALE)))
kmb.cost_cut = VALID_SCORE_CUT
kmb.gscene = gscene

kmm.init_node(timeout=10)

In [5]:
pole_pt_list, pole_res = kmm.update_map(gscene, crob, MOBILE_BASE, timeout=100)
if not CONNECT_MOBILE:
    crob.joint_move_make_sure(kmm.Q_map)

[WARN] Vertices for mesh should be have center point (0,0,0). Auto adjusting.


In [6]:

# pole_list = kmm.add_pixel_poles("obs_pt", gscene, pole_pt_list, pole_res)
# gcheck.ignore_always = pole_list

# gscene.update_markers_all() 

In [7]:
indy.move_joint_s_curve(crob.home_pose[6:], N_div=500)

Connect: Server IP (192.168.0.3)
Move Tool: -0.0869937942786 (1639572379.83)
[KTOOL] send degree: 0.0
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 0, 0)
[KTOOL] recv �  �
Move Tool: -0.177419220424 (1639572379.95)
[KTOOL] send degree: -0.0869937942786
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 0, 0)
[KTOOL] recv �  �
Move Tool: -0.299566255323 (1639572380.08)
[KTOOL] send degree: -0.177419220424
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 255)
[KTOOL] recv �  �
Move Tool: -0.481929652677 (1639572380.21)
[KTOOL] send degree: -0.299566255323
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 254)
[KTOOL] recv �  �
Move Tool: -0.672151578973 (1639572380.34)
[KTOOL] send degree: -0.481929652677
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 252)
[KTOOL] recv �  �
Move Tool: -0.933259747109 (1639572380.46)
[KTOOL] send degree: -0.672151578973
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 250)
[KTOOL] recv �  �
M

Move Tool: -34.3164291741 (1639572386.32)
[KTOOL] send degree: -33.5863651848
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 177)
[KTOOL] recv �  �
Move Tool: -35.029701371 (1639572386.44)
[KTOOL] send degree: -34.3164291741
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 169)
[KTOOL] recv �  �
Move Tool: -35.7251681766 (1639572386.56)
[KTOOL] send degree: -35.029701371
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 162)
[KTOOL] recv �  �
Move Tool: -36.4018412945 (1639572386.68)
[KTOOL] send degree: -35.7251681766
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 155)
[KTOOL] recv �  �
Move Tool: -37.0587591353 (1639572386.8)
[KTOOL] send degree: -36.4018412945
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 148)
[KTOOL] recv �  �
Move Tool: -37.7989510213 (1639572386.93)
[KTOOL] send degree: -37.0587591353
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 142)
[KTOOL] recv �  �
Move Tool: -38.4099025767 (1639

In [8]:
Qcur = crob.get_real_robot_pose()

## 1. Prepare scene

In [9]:
TABLE_DIMS = (0.60,1.2, 0.74)
TABLE_LOC = (0,1.85,TABLE_DIMS[2]/2)
TABLE_ANG = 0
TABLE_MARGIN_I = 0.03
TABLE_MARGIN_O = 0.03
CLEARANCE = 0.002
TABLE_COLOR = (0.8,0.8,0.8,1)
SURF_COLOR = (0,1,0,0.4)
COL_COLOR = (1,1,1,0.2)

gscene.create_safe(GEOTYPE.BOX, "table", link_name="base_link",
                   dims=TABLE_DIMS, center=TABLE_LOC, rpy=Rot2rpy(Rot_axis(3, TABLE_ANG)),
                   color=TABLE_COLOR, collision=False
                  )
gscene.create_safe(GEOTYPE.BOX, "table_col", link_name="base_link",
                   dims=np.add(TABLE_DIMS, 
                               (TABLE_MARGIN_O*2, TABLE_MARGIN_O*2, 0)), 
                   center=(0,0,0), rpy=(0,0,0), color=COL_COLOR, 
                   collision=True, parent="table"
                  )
table_surf = gscene.create_safe(GEOTYPE.BOX, "table_surf", link_name="base_link",
                   dims=np.subtract(TABLE_DIMS[:2]+(CLEARANCE,), 
                                    (TABLE_MARGIN_I*2, TABLE_MARGIN_I*2, 0)), 
                   center=(0,0,(TABLE_DIMS[2]+CLEARANCE)/2), 
                   rpy=(0,0,0), color=SURF_COLOR, parent="table",
                   collision=False
                  )

In [10]:
SOFA_DIMS = (0.49,1.0, 0.43)
SOFA_LOC = (0.2,0.7,SOFA_DIMS[2]/2)
SOFA_SIDE_THIC = 0.2
SOFA_SIDE_HEIT = 0.54
SOFA_ANG = 0

SOFA_MARGIN_O = 0.03
SOFA_MARGIN_I = 0.05
SOFA_UV_DIST = 0.15

SOFA_COLOR = (0.4,0.4,0.4,1)

sofa = gscene.create_safe(GEOTYPE.BOX, "sofa", link_name="base_link",
                   dims=SOFA_DIMS, center=SOFA_LOC, 
                   rpy=Rot2rpy(Rot_axis(3, SOFA_ANG)),
                   color=SOFA_COLOR, collision=False
                  )
gscene.create_safe(GEOTYPE.BOX, "sofa_col", link_name="base_link",
                   dims=np.add(SOFA_DIMS, SOFA_MARGIN_O*2), 
                   center=(0,0,0), rpy=(0,0,0),
                   color=COL_COLOR, collision=True, parent="sofa"
                  )
# sofa_left = gscene.create_safe(GEOTYPE.BOX, "sofa_left", link_name="base_link",
#                    dims=(SOFA_DIMS[0], SOFA_SIDE_THIC, SOFA_DIMS[2]+SOFA_SIDE_HEIT), 
#                    center=(0,(SOFA_DIMS[1]+SOFA_SIDE_THIC)/2, SOFA_SIDE_HEIT/2), 
#                    rpy=(0,0,0), color=SOFA_COLOR, collision=False, parent="sofa"
#                   )
# gscene.create_safe(GEOTYPE.BOX, "sofa_left_col", link_name="base_link",
#                    dims=np.add(sofa_left.dims, SOFA_MARGIN_O*2), 
#                    center=(0,0,0), rpy=(0,0,0),
#                    color=COL_COLOR, collision=True, parent="sofa_left"
#                   )
# sofa_right = gscene.create_safe(GEOTYPE.BOX, "sofa_right", link_name="base_link",
#                    dims=(SOFA_DIMS[0], SOFA_SIDE_THIC, SOFA_DIMS[2]+SOFA_SIDE_HEIT), 
#                    center=(0,-(SOFA_DIMS[1]+SOFA_SIDE_THIC)/2, SOFA_SIDE_HEIT/2), 
#                    rpy=(0,0,0), color=SOFA_COLOR, collision=False, parent="sofa"
#                   )
# gscene.create_safe(GEOTYPE.BOX, "sofa_right_col", link_name="base_link",
#                    dims=np.add(sofa_right.dims, SOFA_MARGIN_O*2), 
#                    center=(0,0,0), rpy=(0,0,0),
#                    color=COL_COLOR, collision=True, parent="sofa_right"
#                   )
sofa_back = gscene.create_safe(GEOTYPE.BOX, "sofa_back", link_name="base_link",
#                    dims=(SOFA_SIDE_THIC, SOFA_DIMS[1]+SOFA_SIDE_THIC*2, SOFA_DIMS[2]+SOFA_SIDE_HEIT), 
                   dims=(SOFA_SIDE_THIC, SOFA_DIMS[1], SOFA_DIMS[2]+SOFA_SIDE_HEIT), 
                   center=(-(SOFA_DIMS[0]+SOFA_SIDE_THIC)/2, 0, SOFA_SIDE_HEIT/2), 
                   rpy=(0,0,0), color=SOFA_COLOR, collision=False, parent="sofa"
                  )
gscene.create_safe(GEOTYPE.BOX, "sofa_back_col", link_name="base_link",
                   dims=np.add(sofa_back.dims, SOFA_MARGIN_O*2), 
                   center=(0,0,0), rpy=(0,0,0),
                   color=COL_COLOR, collision=True, parent="sofa_back"
                  )

sofa_surf = gscene.create_safe(GEOTYPE.BOX, "sofa_surf", link_name="base_link",
                               dims=tuple(np.subtract(SOFA_DIMS[:2], SOFA_MARGIN_I*2))+(CLEARANCE,), 
                               center=(0,0,SOFA_DIMS[2]/2+SOFA_UV_DIST), collision=False, 
                               rpy=(0,0,0), color=SURF_COLOR, parent="sofa"
                              )

sofa_back_surf = gscene.create_safe(GEOTYPE.BOX, "sofa_back_surf", link_name="base_link",
                               dims=sofa_back.dims[:2]+(CLEARANCE,), collision=False, 
                               center=(SOFA_MARGIN_I*2.5,0,sofa_back.dims[2]/2+SOFA_MARGIN_O+CLEARANCE), 
                               rpy=(0,0,0), color=SURF_COLOR, parent="sofa_back"
                              )

# sofa_left_surf = gscene.create_safe(GEOTYPE.BOX, "sofa_left_surf", link_name="base_link",
#                                dims=sofa_left.dims[:2]+(CLEARANCE,), collision=False, 
#                                center=(0,0,sofa_left.dims[2]/2+SOFA_UV_DIST), 
#                                rpy=(0,0,0), color=SURF_COLOR, parent="sofa_left"
#                               )

# sofa_right_surf = gscene.create_safe(GEOTYPE.BOX, "sofa_right_surf", link_name="base_link",
#                                dims=sofa_right.dims[:2]+(CLEARANCE,), collision=False, 
#                                center=(0,0,sofa_right.dims[2]/2+SOFA_UV_DIST), 
#                                rpy=(0,0,0), color=SURF_COLOR, parent="sofa_right"
#                               )

sofa_front = gscene.create_safe(GEOTYPE.BOX, "sofa_front", link_name="base_link",
                               dims=(sofa.dims[2], sofa.dims[1])+(CLEARANCE,), 
                               center=(sofa.dims[0]/2+SOFA_UV_DIST,0,SOFA_MARGIN_I), collision=False, 
                               rpy=(0,np.pi/2,0), color=SURF_COLOR, parent="sofa"
                              )

# sofa_left_front = gscene.create_safe(GEOTYPE.BOX, "sofa_left_front", link_name="base_link",
#                                dims=(sofa_left.dims[2], sofa_left.dims[1])+(CLEARANCE,), 
#                                center=(sofa_left.dims[0]/2+SOFA_UV_DIST,0,+SOFA_MARGIN_I), collision=False, 
#                                rpy=(0,np.pi/2,0), color=SURF_COLOR, parent="sofa_left"
#                               )

# sofa_right_front = gscene.create_safe(GEOTYPE.BOX, "sofa_right_front", link_name="base_link",
#                                dims=(sofa_right.dims[2], sofa_right.dims[1])+(CLEARANCE,), 
#                                center=(sofa_right.dims[0]/2+SOFA_UV_DIST,0,+SOFA_MARGIN_I), collision=False, 
#                                rpy=(0,np.pi/2,0), color=SURF_COLOR, parent="sofa_right"
#                               )

sofa_back_front = gscene.create_safe(GEOTYPE.BOX, "sofa_back_front", link_name="base_link",
#                                dims=(sofa_back.dims[2]-SOFA_DIMS[2]-SOFA_MARGIN_I, sofa_back.dims[1]-SOFA_SIDE_THIC*2-SOFA_MARGIN_I*3)+(CLEARANCE,), 
                               dims=(sofa_back.dims[2]-SOFA_DIMS[2]-SOFA_MARGIN_I, sofa_back.dims[1]-SOFA_MARGIN_I*3)+(CLEARANCE,), 
                               center=(sofa_back.dims[0]/2+SOFA_UV_DIST,0,SOFA_DIMS[2]/2+SOFA_MARGIN_I*1.5), 
                               rpy=(0,np.pi/2,0), color=SURF_COLOR, collision=False, parent="sofa_back"
                              )

### Set UI

In [11]:
from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production


### 1.0 Wait task start queue

In [12]:
if CONNECT_TASK_PLANNER:
    while servicer.object_info_running.object_id < 0:
        time.sleep(1)

   Use a production WSGI server instead.
 * Debug mode: off



## 2. Prepare cleaning

In [13]:
from pkg.planning.constraint.constraint_common import *
from pkg.planning.constraint.constraint_actor import *
from pkg.planning.constraint.constraint_subject import *
from pkg.utils.code_scraps import get_look_motion

Exception in thread Thread-18:
Traceback (most recent call last):
  File "/usr/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/usr/lib/python2.7/threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "/home/kiro-ros/.local/lib/python2.7/site-packages/dash/dash.py", line 1716, in run_server
    self.server.run(host=host, port=port, debug=debug, **flask_run_options)
  File "/home/kiro-ros/.local/lib/python2.7/site-packages/flask/app.py", line 990, in run
    run_simple(host, port, self, **options)
  File "/home/kiro-ros/.local/lib/python2.7/site-packages/werkzeug/serving.py", line 1052, in run_simple
    inner()
  File "/home/kiro-ros/.local/lib/python2.7/site-packages/werkzeug/serving.py", line 1005, in inner
    fd=fd,
  File "/home/kiro-ros/.local/lib/python2.7/site-packages/werkzeug/serving.py", line 848, in make_server
    host, port, app, request_handler, passthrough_errors, ssl_context, fd=fd
  File "/home/kiro-r

In [14]:
# mplan.reset_log(flag_log=True)
Qcur = crob.get_real_robot_pose()
Qcur[-1] = np.deg2rad(-45)
crob.joint_move_make_sure(Qcur)
HOME_POSE_SWEEP = np.copy(Qcur)
# HOME_POSE_SWEEP[6:] = 0
crob.home_pose = HOME_POSE_SWEEP
crob.home_dict = list2dict(crob.home_pose, 
                           gscene.joint_names)
floor_ws = gscene.NAME_DICT["floor_ws"]    

VEL_LIMS = 0.4
ACC_LIMS = 0.4
SWP_VEL_LIMS = 0.2
SWP_ACC_LIMS = 0.2
RADI_DEG = 1

if CONNECT_INDY:
    indy.QVEL_LEVEL = 3
#     indy.collision_policy = indy_trajectory_client_nosdk.POLICY_NO_COLLISION_DETECTION
    indy.reset()

gtimer = GlobalTimer()
    
mode_switcher=ModeSwitcherLED(pscene, robot_name=ROBOT_NAME, brush_face=brush_face.geometry)

def no_offset(gxter, crob, mplan, robot_name, Qref):
    return Qref, Qref

Send to mobile: [ 0.953  1.817  1.     0.032]
             -> [ 0.953  1.817  0.999  0.047]
Distance=0.0299999665805 ([ 0.    0.   -0.03])
[WARN] TOO SMALL VARIANCE, REDUCE TIMEOUT to 3.75
goal reach: 1 -> 0 (1639571893.39)
goal reach: 0 -> 1 (1639571896.29)
goal reach signal edge up
End up at=[ 0.919  1.802  2.979] (0.0793 / 0.0793)
Connect: Server IP (192.168.0.3)
Connect: Server IP (192.168.0.3)
Move Tool: -45.0 (1639571898.54)
[KTOOL] send degree: -45.0
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 62)
[KTOOL] recv �  �
Move Tool: -45.0 (1639571898.81)
[KTOOL] send degree: -45.0
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 62)
[KTOOL] recv �  �
Connect: Server IP (192.168.0.3)


## 3. Table cleaning

In [15]:
T_e_brush = brush_face.get_tf_handle(crob.home_dict, from_link=TIP_LINK)
T_brush_e = SE3_inv(T_e_brush)

In [16]:
EE_HEIGHT = round(table_surf.get_tf(HOME_DICT)[2,3] + table_surf.dims[2]/2, 5) \
                + T_brush_e[2, 3] - INDY_BASE_OFFSET[2]
gtimer.reset(scale=1, timeunit='s', stack=True)
gxter = GreedyExecuter(ppline, brush_face, TOOL_DIM, Qcur, 
                       vel_lims=VEL_LIMS, acc_lims=ACC_LIMS, 
                       swp_vel_lims=SWP_VEL_LIMS, swp_acc_lims=SWP_ACC_LIMS
                      )
tplan.node_trial_max = NODE_TRIAL_MAX

gxter.set_test_kwargs(multiprocess=PLANNING_MULTIPROC, N_agents=N_AGENTS,
                      timeout=TIMEOUT_MOTION, timeout_loop=TIMEOUT_FULL, 
                      verbose=VERBOSE, max_solution_count=MAX_SOL_NUM)

gxter.get_division_dict(table_surf, "front", "X", EE_HEIGHT, div_num=(4,4))
gxter.init_base_divs(Qcur)
snode_schedule_list_d, Qcur, covereds = gxter.greedy_execute(
    Qcur, 1, mode_switcher, no_offset, cost_cut=BASE_COST_CUT)

gxter.test_clear()
gscene.clear_virtuals()

('Height Reference: ', 0.46399999845027923)
Send to mobile: [ 0.918  1.796  0.997  0.078]
             -> [ 0.932  1.815  1.     0.   ]
Distance=0.158254171566 ([ 0.014  0.019  0.157])
goal reach: 1 -> 0 (1639571902.89)
goal reach: 0 -> 1 (1639571905.89)
goal reach signal edge up
End up at=[ 0.957  1.841 -3.123] (6.26 / 6.26)
[PLAN] Adjust base once. ((0.93200000000000005, -0.035000000000000003, -0.74099999999999999), (0.0, 0.0, 1.0, 0.0)) / ((0.93000000000000005, -0.029999999999999999, -0.73999999999999999), (1.0, 0.0))
[PLAN] Qcur: [ 0.932  1.815  3.142]
[PLAN] Qref: [ 0.932  1.815  3.142]
[PLAN] tar: [ 0.932  1.815  3.142]
Send to mobile: [ 0.957  1.841  1.    -0.009]
             -> [ 0.932  1.815  1.     0.   ]
Distance=0.040646888316 ([-0.025 -0.026 -0.019])
goal reach: 1 -> 0 (1639571907.29)
goal reach: 0 -> 1 (1639571909.09)
goal reach signal edge up
End up at=[ 0.921  1.809 -3.131] (6.27 / 6.27)
[PLAN] Line idc [0, 4, 8, 12]
[PLAN] Try idc (0, 4, 8, 12)
Use 10/12 agents
Goal r

Goal reached
======================= terminated 6: required answers acquired  (0.6/5.0) ===============================
Goal reached
======================= terminated 8: required answers acquired  (0.7/5.0) ===============================
======================= terminated 4: required answers acquired  (0.7/5.0) ===============================
Goal reached
Goal reached
======================= terminated 7: required answers acquired  (0.7/5.0) ===============================
======================= terminated 3: required answers acquired  (0.7/5.0) ===============================
Goal reached
======================= terminated 2: required answers acquired  (0.7/5.0) ===============================
======================= terminated 9: Stop called from other agent  (0.7/5.0) ===============================
Goal reached
======================= terminated 1: required answers acquired  (0.7/5.0) ===============================
======================= terminated 0: Stop called from other ag

Move Tool: -39.2763407692 (1639571919.09)
[KTOOL] send degree: -39.455393995
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 118)
[KTOOL] recv �  �
Move Tool: -39.0674544464 (1639571919.22)
[KTOOL] send degree: -39.2763407692
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 120)
[KTOOL] recv �  �
Move Tool: -38.858580786 (1639571919.36)
[KTOOL] send degree: -39.0674544464
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 122)
[KTOOL] recv �  �
Move Tool: -38.6795587358 (1639571919.49)
[KTOOL] send degree: -38.858580786
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 124)
[KTOOL] recv �  �
Move Tool: -38.4707163506 (1639571919.62)
[KTOOL] send degree: -38.6795587358
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 126)
[KTOOL] recv �  �
Move Tool: -38.2917249412 (1639571919.74)
[KTOOL] send degree: -38.4707163506
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 128)
[KTOOL] recv �  �
Move Tool: -38.0829243121 (1639

Move Tool: -29.8033738134 (1639571926.36)
[KTOOL] send degree: -29.9044221974
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 213)
[KTOOL] recv �  �
Move Tool: -29.6827730417 (1639571926.49)
[KTOOL] send degree: -29.8033738134
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 214)
[KTOOL] recv �  �
Move Tool: -29.5832286882 (1639571926.62)
[KTOOL] send degree: -29.6827730417
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 216)
[KTOOL] recv �  �
Move Tool: -29.4831810685 (1639571926.75)
[KTOOL] send degree: -29.5832286882
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 217)
[KTOOL] recv �  �
Move Tool: -29.366484571 (1639571926.87)
[KTOOL] send degree: -29.4831810685
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 218)
[KTOOL] recv �  �
Move Tool: -29.267402434 (1639571927.01)
[KTOOL] send degree: -29.366484571
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 219)
[KTOOL] recv �  �
Move Tool: -29.1513802173 (1639

Move Tool: -34.6190307783 (1639571933.22)
[KTOOL] send degree: -34.4575510303
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 168)
[KTOOL] recv �  �
Move Tool: -34.7588044332 (1639571933.35)
[KTOOL] send degree: -34.6190307783
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 166)
[KTOOL] recv �  �
Move Tool: -34.8680977036 (1639571933.49)
[KTOOL] send degree: -34.7588044332
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 165)
[KTOOL] recv �  �
Move Tool: -34.8899260296 (1639571933.63)
[KTOOL] send degree: -34.8680977036
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 164)
[KTOOL] recv �  �
Move Tool: -34.8071040584 (1639571933.76)
[KTOOL] send degree: -34.8899260296
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 164)
[KTOOL] recv �  �
Move Tool: -34.6524226169 (1639571933.91)
[KTOOL] send degree: -34.8071040584
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 164)
[KTOOL] recv �  �
Move Tool: -34.4647479508 (1

...

Move Tool: -37.7996805113 (1639572067.55)
[KTOOL] send degree: -37.5378975691
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 137)
[KTOOL] recv �  �
Move Tool: -38.0418628435 (1639572067.69)
[KTOOL] send degree: -37.7996805113
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 135)
[KTOOL] recv �  �
Move Tool: -38.1672989065 (1639572067.82)
[KTOOL] send degree: -38.0418628435
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 132)
[KTOOL] recv �  �
Move Tool: -38.174545845 (1639572067.87)
[KTOOL] send degree: -38.1672989065
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 131)
[KTOOL] recv �  �
LED ON
[KTOOL] led_on
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.LED_ON: 4>)
[KTOOL] recv �  �
Move Tool: -38.1745486811 (1639572068.62)
[KTOOL] send degree: -38.174545845
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 131)
[KTOOL] recv �  �
Move Tool: -38.1745486811 (1639572068.75)
[KTOOL] send degree: -38.1745486811
[KTOOL] send (<MOT

Move Tool: -38.1745486811 (1639572075.05)
[KTOOL] send degree: -38.1745486811
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 131)
[KTOOL] recv �  �
Move Tool: -38.1745486811 (1639572075.18)
[KTOOL] send degree: -38.1745486811
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 131)
[KTOOL] recv �  �
Move Tool: -38.1745486811 (1639572075.31)
[KTOOL] send degree: -38.1745486811
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 131)
[KTOOL] recv �  �
Move Tool: -38.1745486811 (1639572075.43)
[KTOOL] send degree: -38.1745486811
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 131)
[KTOOL] recv �  �
Move Tool: -38.1745486811 (1639572075.56)
[KTOOL] send degree: -38.1745486811
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 131)
[KTOOL] recv �  �
Move Tool: -38.1745486811 (1639572075.69)
[KTOOL] send degree: -38.1745486811
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 131)
[KTOOL] recv �  �
Move Tool: -38.1745486811 (1

Move Tool: -38.1745486811 (1639572082.01)
[KTOOL] send degree: -38.1745486811
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 131)
[KTOOL] recv �  �
Move Tool: -38.1745486811 (1639572082.16)
[KTOOL] send degree: -38.1745486811
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 131)
[KTOOL] recv �  �
Move Tool: -38.1745486811 (1639572082.3)
[KTOOL] send degree: -38.1745486811
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 131)
[KTOOL] recv �  �
Move Tool: -38.1745486811 (1639572082.44)
[KTOOL] send degree: -38.1745486811
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 131)
[KTOOL] recv �  �
Move Tool: -38.1745486811 (1639572082.61)
[KTOOL] send degree: -38.1745486811
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 131)
[KTOOL] recv �  �
Move Tool: -38.1745486811 (1639572082.73)
[KTOOL] send degree: -38.1745486811
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 131)
[KTOOL] recv �  �
Move Tool: -38.1745486811 (16

Move Tool: -38.1745486811 (1639572089.1)
[KTOOL] send degree: -38.1745486811
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 131)
[KTOOL] recv �  �
Move Tool: -38.1745486811 (1639572089.23)
[KTOOL] send degree: -38.1745486811
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 131)
[KTOOL] recv �  �
Move Tool: -38.1745486811 (1639572089.36)
[KTOOL] send degree: -38.1745486811
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 131)
[KTOOL] recv �  �
Move Tool: -38.1745486811 (1639572089.52)
[KTOOL] send degree: -38.1745486811
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 131)
[KTOOL] recv �  �
Move Tool: -38.1745486811 (1639572089.65)
[KTOOL] send degree: -38.1745486811
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 131)
[KTOOL] recv �  �
Move Tool: -38.1745486811 (1639572089.77)
[KTOOL] send degree: -38.1745486811
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 131)
[KTOOL] recv �  �
Move Tool: -38.1745486811 (16

AssertionError: MOVE robot to trajectory initial: current robot pose does not match with trajectory initial state

## 4. Sofa cleaning

### 4.1 upside

In [ ]:
EE_HEIGHT = round(sofa_surf.get_tf(HOME_DICT)[2,3] + sofa_surf.dims[2]/2, 5) \
                + T_brush_e[2, 3] - INDY_BASE_OFFSET[2]
gtimer.reset(scale=1, timeunit='s', stack=True)
gxter = GreedyExecuter(ppline, brush_face, TOOL_DIM, Qcur, 
                       vel_lims=VEL_LIMS, acc_lims=ACC_LIMS, 
                       swp_vel_lims=SWP_VEL_LIMS, swp_acc_lims=SWP_ACC_LIMS
                      )
tplan.node_trial_max = NODE_TRIAL_MAX

gxter.set_test_kwargs(multiprocess=PLANNING_MULTIPROC, N_agents=N_AGENTS,
                      timeout=TIMEOUT_MOTION, timeout_loop=TIMEOUT_FULL, 
                      verbose=VERBOSE, max_solution_count=MAX_SOL_NUM)

gxter.get_division_dict(sofa_surf, "front", "X", EE_HEIGHT)
gxter.init_base_divs(Qcur)
# gxter.mark_tested(None, None, covereds_all, [])
snode_schedule_list_d, Qcur, covereds = gxter.greedy_execute(
    Qcur, 1, mode_switcher, no_offset, cost_cut=BASE_COST_CUT)

gxter.test_clear()
print(gtimer)
gscene.clear_virtuals()

In [ ]:
EE_HEIGHT = round(sofa_back_surf.get_tf(HOME_DICT)[2,3] + sofa_back_surf.dims[2]/2, 5) \
                + T_brush_e[2, 3] - INDY_BASE_OFFSET[2]
gtimer.reset(scale=1, timeunit='s', stack=True)
gxter = GreedyExecuter(ppline, brush_face, TOOL_DIM, Qcur, 
                       vel_lims=VEL_LIMS, acc_lims=ACC_LIMS, 
                       swp_vel_lims=SWP_VEL_LIMS, swp_acc_lims=SWP_ACC_LIMS
                      )
tplan.node_trial_max = NODE_TRIAL_MAX

gxter.set_test_kwargs(multiprocess=PLANNING_MULTIPROC, N_agents=N_AGENTS,
                      timeout=TIMEOUT_MOTION, timeout_loop=TIMEOUT_FULL, 
                      verbose=VERBOSE, max_solution_count=MAX_SOL_NUM)

gxter.get_division_dict(sofa_back_surf, "front", "X", EE_HEIGHT)
gxter.init_base_divs(Qcur)
# gxter.mark_tested(None, None, covereds_all, [])
snode_schedule_list_d, Qcur, covereds = gxter.greedy_execute(
    Qcur, 1, mode_switcher, no_offset, cost_cut=BASE_COST_CUT)

gxter.test_clear()
print(gtimer)
gscene.clear_virtuals()

In [ ]:
# EE_HEIGHT = round(sofa_right_surf.get_tf(HOME_DICT)[2,3] + sofa_right_surf.dims[2]/2, 5) \
#                 + T_brush_e[2, 3] - INDY_BASE_OFFSET[2]
# gtimer.reset(scale=1, timeunit='s', stack=True)
# gxter = GreedyExecuter(ppline, brush_face, TOOL_DIM, Qcur, 
#                        vel_lims=VEL_LIMS, acc_lims=ACC_LIMS, 
#                        swp_vel_lims=SWP_VEL_LIMS, swp_acc_lims=SWP_ACC_LIMS
#                       )
# tplan.node_trial_max = NODE_TRIAL_MAX

# gxter.set_test_kwargs(multiprocess=PLANNING_MULTIPROC, N_agents=N_AGENTS,
#                       timeout=TIMEOUT_MOTION, timeout_loop=TIMEOUT_FULL, 
#                       verbose=VERBOSE, max_solution_count=MAX_SOL_NUM)

# gxter.get_division_dict(sofa_right_surf, "front", "X", EE_HEIGHT)
# gxter.init_base_divs(Qcur)
# # gxter.mark_tested(None, None, covereds_all, [])
# snode_schedule_list_d, Qcur, covereds = gxter.greedy_execute(
#     Qcur, 1, mode_switcher, no_offset, cost_cut=BASE_COST_CUT)

# gxter.test_clear()
# print(gtimer)
# gscene.clear_virtuals()

In [ ]:
# EE_HEIGHT = round(sofa_left_surf.get_tf(HOME_DICT)[2,3] + sofa_left_surf.dims[2]/2, 5) \
#                 + T_brush_e[2, 3] - INDY_BASE_OFFSET[2]
# gtimer.reset(scale=1, timeunit='s', stack=True)
# gxter = GreedyExecuter(ppline, brush_face, TOOL_DIM, Qcur, 
#                        vel_lims=VEL_LIMS, acc_lims=ACC_LIMS, 
#                        swp_vel_lims=SWP_VEL_LIMS, swp_acc_lims=SWP_ACC_LIMS
#                       )
# tplan.node_trial_max = NODE_TRIAL_MAX

# gxter.set_test_kwargs(multiprocess=PLANNING_MULTIPROC, N_agents=N_AGENTS,
#                       timeout=TIMEOUT_MOTION, timeout_loop=TIMEOUT_FULL, 
#                       verbose=VERBOSE, max_solution_count=MAX_SOL_NUM)

# gxter.get_division_dict(sofa_left_surf, "front", "X", EE_HEIGHT)
# gxter.init_base_divs(Qcur)
# # gxter.mark_tested(None, None, covereds_all, [])
# snode_schedule_list_d, Qcur, covereds = gxter.greedy_execute(
#     Qcur, 1, mode_switcher, no_offset, cost_cut=BASE_COST_CUT)

# gxter.test_clear()
# print(gtimer)
# gscene.clear_virtuals()

In [19]:
gtimer.reset(scale=1, timeunit='s', stack=True)
gxter = GreedyExecuter(ppline, brush_face, TOOL_DIM, Qcur, 
                       vel_lims=VEL_LIMS, acc_lims=ACC_LIMS, 
                       swp_vel_lims=SWP_VEL_LIMS, swp_acc_lims=SWP_ACC_LIMS
                      )
tplan.node_trial_max = NODE_TRIAL_MAX

gxter.set_test_kwargs(multiprocess=PLANNING_MULTIPROC, N_agents=N_AGENTS,
                      timeout=TIMEOUT_MOTION, timeout_loop=TIMEOUT_FULL, 
                      verbose=VERBOSE, max_solution_count=MAX_SOL_NUM)

gxter.get_division_dict(sofa_back_front, "down", "Z", None)
gxter.init_base_divs(Qcur)
# gxter.mark_tested(None, None, covereds_all, [])
snode_schedule_list_d, Qcur, covereds = gxter.greedy_execute(
    Qcur, -1, mode_switcher, no_offset, cost_cut=BASE_COST_CUT)

gxter.test_clear()
print(gtimer)
gscene.clear_virtuals()

Send to mobile: [ 1.065  0.623  1.     0.018]
             -> [ 1.169  0.59   1.     0.   ]
Distance=0.114595331571 ([ 0.104 -0.033  0.037])
goal reach: 1 -> 0 (1639567684.77)
goal reach: 0 -> 1 (1639567688.17)
goal reach signal edge up
End up at=[ 1.18   0.585 -3.105] (6.25 / 6.25)
[PLAN] Adjust base once. ((0.77500000000000002, -0.11, 1.0640000000000001), (0.70699999999999996, -0.0, -0.70699999999999996, 0.0)) / ((0.77000000000000002, -0.11, 1.0600000000000001), (0.70999999999999996, 0.0))
[PLAN] Qcur: [ 1.169  0.59   3.142]
[PLAN] Qref: [ 1.169  0.59   3.142]
[PLAN] tar: [ 1.169  0.59   3.142]
Send to mobile: [ 1.18   0.585  1.    -0.018]
             -> [ 1.169  0.59   1.     0.   ]
Distance=0.0385795079542 ([-0.011  0.005 -0.037])
[WARN] TOO SMALL VARIANCE, REDUCE TIMEOUT to 4.82
goal reach: 1 -> 0 (1639567689.57)
goal reach: 0 -> 1 (1639567690.87)
goal reach signal edge up
End up at=[ 1.158  0.579 -3.135] (6.28 / 6.28)
[PLAN] Line idc [0, 3]
[PLAN] Try idc (0, 3)
Use 10/12 agents

======================= terminated 8: node queue empty 4th  (1.5/5.0) ===============================
======================= terminated 6: node queue empty 4th  (1.5/5.0) ===============================
======================= terminated 9: node queue empty 3th  (1.5/5.0) ===============================
======================= terminated 0: node queue empty 6th  (1.5/5.0) ===============================
======================= terminated 4: node queue empty 5th  (1.5/5.0) ===============================
======================= terminated 3: node queue empty 7th  (1.5/5.0) ===============================
======================= terminated 2: node queue empty 8th  (1.6/5.0) ===============================
======================= terminated 7: node queue empty 9th  (1.7/5.0) ===============================
========================== FINISHED (5.1 / 5.0 s) ==============================]
[PLAN] Line idc [1, 4]
[PLAN] Try idc (1, 4)
Use 10/12 agents
Goal reached
Goal reached
Goal reached
G

======================= terminated 9: Stop called from other agent  (0.7/5.0) ===============================
Goal reached
======================= terminated 1: required answers acquired  (0.7/5.0) ===============================
Goal reached
======================= terminated 5: required answers acquired  (0.8/5.0) ===============================
======================= terminated 8: Stop called from other agent  (0.8/5.0) ===============================
========================== FINISHED (0.8 / 5.0 s) ==============================]
Goal reached
Move Tool: -44.9945616082
[KTOOL] send degree: -45.0
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 62)
[KTOOL] recv �  �
Move Tool: -44.9171887849
[KTOOL] send degree: -44.9945616082
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 63)
[KTOOL] recv �  �
Move Tool: -44.7592751908
[KTOOL] send degree: -44.9171887849
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 63)
[KTOOL] recv �  �
Move Tool: -44.6162

Move Tool: -30.9451299444
[KTOOL] send degree: -31.0735764852
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 202)
[KTOOL] recv �  �
Move Tool: -30.8536239623
[KTOOL] send degree: -30.9451299444
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 203)
[KTOOL] recv �  �
Move Tool: -30.7058016936
[KTOOL] send degree: -30.8536239623
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 204)
[KTOOL] recv �  �
Move Tool: -30.6013002258
[KTOOL] send degree: -30.7058016936
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 205)
[KTOOL] recv �  �
Move Tool: -30.4928972382
[KTOOL] send degree: -30.6013002258
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 206)
[KTOOL] recv �  �
Move Tool: -30.3245701379
[KTOOL] send degree: -30.4928972382
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 208)
[KTOOL] recv �  �
Move Tool: -30.2095851171
[KTOOL] send degree: -30.3245701379
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 209)

Move Tool: -22.9427991642
[KTOOL] send degree: -23.0555330248
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 26)
[KTOOL] recv �  �
Move Tool: -22.773672166
[KTOOL] send degree: -22.9427991642
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 27)
[KTOOL] recv �  �
Move Tool: -22.604609732
[KTOOL] send degree: -22.773672166
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 29)
[KTOOL] recv �  �
Move Tool: -22.491952823
[KTOOL] send degree: -22.604609732
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 30)
[KTOOL] recv �  �
Move Tool: -22.3793476678
[KTOOL] send degree: -22.491952823
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 32)
[KTOOL] recv �  �
Move Tool: -22.2105585261
[KTOOL] send degree: -22.3793476678
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 33)
[KTOOL] recv �  �
Move Tool: -22.0981268182
[KTOOL] send degree: -22.2105585261
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 34)
[KTOOL] recv

...

Move Tool: -23.3444242747
[KTOOL] send degree: -23.0694105487
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 26)
[KTOOL] recv �  �
Move Tool: -23.5271889088
[KTOOL] send degree: -23.3444242747
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 23)
[KTOOL] recv �  �
Move Tool: -23.8006553853
[KTOOL] send degree: -23.5271889088
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 21)
[KTOOL] recv �  �
Move Tool: -23.9825762962
[KTOOL] send degree: -23.8006553853
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 18)
[KTOOL] recv �  �
Move Tool: -24.2547707506
[KTOOL] send degree: -23.9825762962
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 17)
[KTOOL] recv �  �
Move Tool: -24.4357897377
[KTOOL] send degree: -24.2547707506
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 14)
[KTOOL] recv �  �
Move Tool: -24.6165176759
[KTOOL] send degree: -24.4357897377
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 255, 12)
[KTOOL

Move Tool: -31.575442627
[KTOOL] send degree: -31.4950931892
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 198)
[KTOOL] recv �  �
Move Tool: -31.6556372243
[KTOOL] send degree: -31.575442627
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 197)
[KTOOL] recv �  �
Move Tool: -31.7757542147
[KTOOL] send degree: -31.6556372243
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 196)
[KTOOL] recv �  �
Move Tool: -31.8558101773
[KTOOL] send degree: -31.7757542147
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 195)
[KTOOL] recv �  �
Move Tool: -31.9761278236
[KTOOL] send degree: -31.8558101773
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 194)
[KTOOL] recv �  �
Move Tool: -32.0966850404
[KTOOL] send degree: -31.9761278236
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 193)
[KTOOL] recv �  �
Move Tool: -32.1771463891
[KTOOL] send degree: -32.0966850404
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 192)
[

Move Tool: -36.5182432268
[KTOOL] send degree: -36.4810454331
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 148)
[KTOOL] recv �  �
Move Tool: -36.580340006
[KTOOL] send degree: -36.5182432268
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 147)
[KTOOL] recv �  �
Move Tool: -36.6249990104
[KTOOL] send degree: -36.580340006
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 147)
[KTOOL] recv �  �
Move Tool: -36.6953380313
[KTOOL] send degree: -36.6249990104
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 146)
[KTOOL] recv �  �
Move Tool: -36.7435488269
[KTOOL] send degree: -36.6953380313
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 146)
[KTOOL] recv �  �
Move Tool: -36.8146072845
[KTOOL] send degree: -36.7435488269
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 145)
[KTOOL] recv �  �
Move Tool: -36.8605965647
[KTOOL] send degree: -36.8146072845
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 144)
[

Move Tool: -39.9634561744
[KTOOL] send degree: -39.9142104616
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 113)
[KTOOL] recv �  �
Move Tool: -40.0126921305
[KTOOL] send degree: -39.9634561744
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 113)
[KTOOL] recv �  �
Move Tool: -40.0864459927
[KTOOL] send degree: -40.0126921305
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 112)
[KTOOL] recv �  �
Move Tool: -40.1355702369
[KTOOL] send degree: -40.0864459927
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 112)
[KTOOL] recv �  �
Move Tool: -40.2092417358
[KTOOL] send degree: -40.1355702369
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 111)
[KTOOL] recv �  �
Move Tool: -40.2583757716
[KTOOL] send degree: -40.2092417358
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 110)
[KTOOL] recv �  �
Move Tool: -40.3075278652
[KTOOL] send degree: -40.2583757716
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 110)

[KTOOL] recv �  �
Move Tool: -43.4759997179
[KTOOL] send degree: -43.4066337364
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 78)
[KTOOL] recv �  �
Move Tool: -43.523145724
[KTOOL] send degree: -43.4759997179
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 78)
[KTOOL] recv �  �
Move Tool: -43.593682601
[KTOOL] send degree: -43.523145724
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 77)
[KTOOL] recv �  �
Move Tool: -43.6400684567
[KTOOL] send degree: -43.593682601
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 77)
[KTOOL] recv �  �
Move Tool: -43.6856109407
[KTOOL] send degree: -43.6400684567
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 76)
[KTOOL] recv �  �
Move Tool: -43.7517147389
[KTOOL] send degree: -43.6856109407
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>, 254, 76)
[KTOOL] recv �  �
Move Tool: -43.7938886699
[KTOOL] send degree: -43.7517147389
[KTOOL] send (<MOTOR_CMD.OP: 2>, <OP_CMD.DEGREE: 6>

In [20]:
# gtimer.reset(scale=1, timeunit='s', stack=True)
# gxter = GreedyExecuter(ppline, brush_face, TOOL_DIM, Qcur, 
#                        vel_lims=VEL_LIMS, acc_lims=ACC_LIMS, 
#                        swp_vel_lims=SWP_VEL_LIMS, swp_acc_lims=SWP_ACC_LIMS
#                       )
# tplan.node_trial_max = NODE_TRIAL_MAX

# gxter.set_test_kwargs(multiprocess=PLANNING_MULTIPROC, N_agents=N_AGENTS,
#                       timeout=TIMEOUT_MOTION, timeout_loop=TIMEOUT_FULL, 
#                       verbose=VERBOSE, max_solution_count=MAX_SOL_NUM)

# gxter.get_division_dict(sofa_right_front, "down", "Z", None)
# gxter.init_base_divs(Qcur)
# # gxter.mark_tested(None, None, covereds_all, [])
# snode_schedule_list_d, Qcur, covereds = gxter.greedy_execute(
#     Qcur, -1, mode_switcher, no_offset, cost_cut=BASE_COST_CUT)

# gxter.test_clear()
# print(gtimer)
# gscene.clear_virtuals()

In [21]:
gtimer.reset(scale=1, timeunit='s', stack=True)
gxter = GreedyExecuter(ppline, brush_face, TOOL_DIM, Qcur, 
                       vel_lims=VEL_LIMS, acc_lims=ACC_LIMS, 
                       swp_vel_lims=SWP_VEL_LIMS, swp_acc_lims=SWP_ACC_LIMS
                      )
tplan.node_trial_max = NODE_TRIAL_MAX

gxter.set_test_kwargs(multiprocess=PLANNING_MULTIPROC, N_agents=N_AGENTS,
                      timeout=TIMEOUT_MOTION, timeout_loop=TIMEOUT_FULL, 
                      verbose=VERBOSE, max_solution_count=MAX_SOL_NUM)

gxter.get_division_dict(sofa_front, "down", "Z", None)
gxter.init_base_divs(Qcur)
# gxter.mark_tested(None, None, covereds_all, [])
snode_schedule_list_d, Qcur, covereds = gxter.greedy_execute(
    Qcur, -1, mode_switcher, no_offset, cost_cut=BASE_COST_CUT)

gxter.test_clear()
print(gtimer)
gscene.clear_virtuals()

Send to mobile: [ 1.153  0.489  1.     0.002]
             -> [ 1.702  0.37   1.     0.   ]
Distance=0.561323252386 ([ 0.549 -0.119  0.004])
goal reach: 1 -> 0 (1639567823.17)
goal reach: 0 -> 1 (1639567834.87)
goal reach signal edge up
End up at=[ 1.705  0.334  3.129] (0.0382 / 0.0382)
[PLAN] Adjust base once. ((0.26500000000000001, -0.33000000000000002, 1.107), (0.70699999999999996, -0.0, -0.70699999999999996, 0.0)) / ((0.27000000000000002, -0.33000000000000002, 1.1100000000000001), (0.70999999999999996, 0.0))
[PLAN] Qcur: [ 1.702  0.37   3.142]
[PLAN] Qref: [ 1.702  0.37   3.142]
[PLAN] tar: [ 1.702  0.37   3.142]
Send to mobile: [ 1.705  0.334  1.     0.006]
             -> [ 1.702  0.37   1.     0.   ]
Distance=0.0382346029546 ([-0.003  0.036  0.013])
[WARN] TOO SMALL VARIANCE, REDUCE TIMEOUT to 4.78
goal reach: 1 -> 0 (1639567836.27)
goal reach: 0 -> 1 (1639567838.57)
goal reach signal edge up
End up at=[ 1.711  0.372  3.127] (0.0173 / 0.0173)
[PLAN] Line idc [0, 3]
[PLAN] Try id

Process Process-137:
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
Process Process-139:
Process Process-133:
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/usr/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
Process Process-138:
    self.run()
Process Process-141:
Process Process-140:
  File "/usr/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
Process Process-135:
Process Process-132:
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
Process Process-134:
  File "/home/kiro-ros/Projects/rnb-planning/src/pkg/planning/pipeline.py", line 204, in __search_loop
Traceback (most recent call last):
Process Process-136:
  File "/usr/lib/python2.7/multiprocessing/process

KeyboardInterrupt: 

  File "/usr/lib/python2.7/dist-packages/numpy/lib/twodim_base.py", line 187, in eye
    m[:M-k].flat[i::M+1] = 1
KeyboardInterrupt


In [ ]:
# gtimer.reset(scale=1, timeunit='s', stack=True)
# gxter = GreedyExecuter(ppline, brush_face, TOOL_DIM, Qcur, 
#                        vel_lims=VEL_LIMS, acc_lims=ACC_LIMS, 
#                        swp_vel_lims=SWP_VEL_LIMS, swp_acc_lims=SWP_ACC_LIMS
#                       )
# tplan.node_trial_max = NODE_TRIAL_MAX

# gxter.set_test_kwargs(multiprocess=PLANNING_MULTIPROC, N_agents=N_AGENTS,
#                       timeout=TIMEOUT_MOTION, timeout_loop=TIMEOUT_FULL, 
#                       verbose=VERBOSE, max_solution_count=MAX_SOL_NUM)

# gxter.get_division_dict(sofa_left_front, "down", "Z", None)
# gxter.init_base_divs(Qcur)
# # gxter.mark_tested(None, None, covereds_all, [])
# snode_schedule_list_d, Qcur, covereds = gxter.greedy_execute(
#     Qcur, -1, mode_switcher, no_offset, cost_cut=BASE_COST_CUT)

# gxter.test_clear()
# print(gtimer)
# gscene.clear_virtuals()

### Finish

In [ ]:
if CONNECT_TASK_PLANNER:
    servicer.mark_task_finished()

* NoSDK 7DoF 구현하기
* 스케줄 세이브 / 로드 / 편집 구현하기
* 플래닝 마치고 동작 시 빨간 하이라이트 없애기 
* 맵 폴 추가